In [1]:
import pandas as pd
import numpy as np

In [14]:
df=pd.read_csv("finaldata.csv")

In [11]:
df["admittime"]=df["admittime"].apply(lambda x:str(x))

In [16]:
pd.to_str(df["admittime"])

AttributeError: module 'pandas' has no attribute 'to_str'

In [467]:
df['startdate'] = pd.to_datetime(df['startdate'], errors='coerce')
df['enddate'] = pd.to_datetime(df['enddate'], errors='coerce')
df['admittime'] = pd.to_datetime(df['admittime'], errors='coerce')
df['dischtime'] = pd.to_datetime(df['dischtime'], errors='coerce')

In [468]:
cols=df.columns

In [469]:
df[cols[3]+'year'] = df[cols[3]].dt.year
df[cols[3]+'month'] = df[cols[3]].dt.month
df[cols[3]+'day'] = df[cols[3]].dt.day
df[cols[3]+'day_of_week'] = df[cols[3]].dt.dayofweek
df[cols[3]+'hour'] = df[cols[3]].dt.hour

In [470]:
df[cols[4]+'year'] = df[cols[4]].dt.year
df[cols[4]+'month'] = df[cols[4]].dt.month
df[cols[4]+'day'] = df[cols[4]].dt.day
df[cols[4]+'day_of_week'] = df[cols[4]].dt.dayofweek
df[cols[4]+'hour'] = df[cols[4]].dt.hour

In [471]:
df[cols[9]+'year'] = df[cols[9]].dt.year
df[cols[9]+'month'] = df[cols[9]].dt.month
df[cols[9]+'day'] = df[cols[9]].dt.day
df[cols[9]+'day_of_week'] = df[cols[9]].dt.dayofweek
df[cols[9]+'hour'] = df[cols[9]].dt.hour

In [472]:
df[cols[10]+'year'] = df[cols[10]].dt.year
df[cols[10]+'month'] = df[cols[10]].dt.month
df[cols[10]+'day'] = df[cols[10]].dt.day
df[cols[10]+'day_of_week'] = df[cols[10]].dt.dayofweek
df[cols[10]+'hour'] = df[cols[10]].dt.hour

In [473]:
max(df['hadm_id'])

199395

In [474]:
cols=df.columns

In [475]:
cols

Index(['row_id_x', 'subject_id', 'hadm_id', 'admittime', 'dischtime',
       'admission_type', 'hospital_expire_flag', 'has_chartevents_data',
       'row_id_y', 'startdate', 'enddate', 'dose_val_rx', 'dose_unit_rx',
       'form_val_disp', 'form_unit_disp', 'opioid_abuse', 'admdiff',
       'dosgdiff', 'maxdays', 'admlV1', 'admlV2', 'admlV3', 'dischlV1',
       'dischlV2', 'dischlV3', 'insV1', 'insV2', 'insV3', 'mrgsV1', 'mrgsV2',
       'mrgsV3', 'diagV1', 'diagV2', 'diagV3', 'drgtV1', 'drgtV2', 'drgtV3',
       'drgV1', 'drgV2', 'drgV3', 'drgnpV1', 'drgnpV2', 'drgnpV3', 'drgngV1',
       'drgngV2', 'drgngV3', 'routeV1', 'routeV2', 'routeV3', 'admittimeyear',
       'admittimemonth', 'admittimeday', 'admittimeday_of_week',
       'admittimehour', 'dischtimeyear', 'dischtimemonth', 'dischtimeday',
       'dischtimeday_of_week', 'dischtimehour', 'startdateyear',
       'startdatemonth', 'startdateday', 'startdateday_of_week',
       'startdatehour', 'enddateyear', 'enddatemonth', 'endd

In [400]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU, Input
from tensorflow.keras.models import Sequential, Model
import pandas as pd
import numpy as np

# Define Generator
def build_generator(input_dim=100, output_dim=48):
    model = Sequential([
        Dense(256, input_dim=input_dim),
        LeakyReLU(alpha=0.2),
        Dense(512),
        LeakyReLU(alpha=0.2),
        Dense(256),
        LeakyReLU(alpha=0.2),
        Dense(output_dim, activation='tanh')  # Output size should match your data features
    ])
    return model

# Define Discriminator
def build_discriminator(input_dim=48):
    model = Sequential([
        Dense(512, input_dim=input_dim),
        LeakyReLU(alpha=0.2),
        Dense(256),
        LeakyReLU(alpha=0.2),
        Dense(1, activation='sigmoid')  # Output is probability of being real
    ])
    return model

# Define GAN
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze the discriminator when training the GAN
    model = Sequential([generator, discriminator])
    return model

# Compile Models
def compile_models(generator, discriminator, gan):
    discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))
    gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

# Normalize Data
def normalize_data(data):
    data_normalized = data.copy()
    min_max = {}
    for column in data.columns:
        if np.issubdtype(data[column].dtype, np.number):
            min_val = data[column].min()
            max_val = data[column].max()
            min_max[column] = (min_val, max_val)
            # Min-max normalization to [0, 1]
            data_normalized[column] = (data[column] - min_val) / (max_val - min_val)
            # Scale to [-1, 1]
            data_normalized[column] = (data_normalized[column] - 0.5) * 2
    return data_normalized, min_max

# Reverse Normalize Data
def reverse_normalize_data(data_normalized, min_max):
    data_reversed = data_normalized.copy()
    for column in data_normalized.columns:
        if column in min_max:
            min_val, max_val = min_max[column]
            data_reversed[column] = (data_normalized[column] / 2) + 0.5
            data_reversed[column] = data_reversed[column] * (max_val - min_val) + min_val
    return data_reversed

# Train GAN
def train_gan(generator, discriminator, gan, data, epochs=25, batch_size=64):
    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, data.shape[0], batch_size)
        real_data = data.iloc[idx].values
        real_labels = np.ones((batch_size, 1))
        noise = np.random.normal(0, 1, (batch_size, input_dim))
        fake_data = generator.predict(noise)
        fake_labels = np.zeros((batch_size, 1))
        
        d_loss_real = discriminator.train_on_batch(real_data, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, input_dim))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
        
        if epoch % 1000 == 0:
            print(f'Epoch {epoch} | D Loss: {d_loss} | G Loss: {g_loss}')
# Step 1: Filter original data for 'opioid_abuse' == 'yes'
opioid_abuse_yes = df[df['opioid_abuse'] == 1]
opioid_abuse_yes=opioid_abuse_yes.drop([cols[0],cols[1],cols[2],cols[3],cols[4],cols[9],cols[10]],axis=1)

# Step 2: Normalize the filtered data
opioid_abuse_yes_normalized, min_max = normalize_data(opioid_abuse_yes)

# Step 3: Define the GAN components
input_dim = 100
output_dim = opioid_abuse_yes_normalized.shape[1]

generator = build_generator(input_dim, output_dim)
discriminator = build_discriminator(output_dim)
gan = build_gan(generator, discriminator)
compile_models(generator, discriminator, gan)

# Step 4: Train the GAN
train_gan(generator, discriminator, gan, opioid_abuse_yes_normalized)

# Step 5: Generate synthetic data
num_samples_to_generate = 4000
noise = np.random.normal(0, 1, (num_samples_to_generate, input_dim))
generated_data_normalized = generator.predict(noise)
generated_data_normalized_df = pd.DataFrame(generated_data_normalized, columns=opioid_abuse_yes.columns)

# Step 6: Reverse normalization on the generated data
generated_data = reverse_normalize_data(generated_data_normalized_df, min_max)

print(generated_data)


C:\Users\Jamaludeen\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Jamaludeen\anaconda3\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step  


C:\Users\Jamaludeen\anaconda3\lib\site-packages\keras\src\backend\tensorflow\trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0 | D Loss: nan | G Loss: [array(nan, dtype=float32), array(nan, dtype=float32)]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
1

In [476]:
generated_data

,admission_type,hospital_expire_flag,has_chartevents_data,row_id_y,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,opioid_abuse,admdiff,...,startdateyear,startdatemonth,startdateday,startdateday_of_week,startdatehour,enddateyear,enddatemonth,enddateday,enddateday_of_week,enddatehour
0,2.619910,0.071423,0.965048,2514856.000,51321884.0,4.272938,2171.296631,30.776081,1.0,35.295570,...,2008.258179,2.550713,1.780574,0.124380,0.0,2000.946899,10.948043,2.368855,5.570255,0.0
1,2.835562,0.036966,0.922353,2230993.500,49811132.0,6.087144,1454.517456,32.827713,1.0,33.724800,...,2007.290527,3.080406,1.844749,0.168674,0.0,2000.933105,10.492146,2.314125,5.261910,0.0
2,2.907156,0.025290,0.979776,3246302.250,51741772.0,2.822199,555.575806,33.118919,1.0,32.817257,...,2007.177734,3.623435,1.530835,0.015057,0.0,2000.611206,10.907797,1.276327,5.095401,0.0
3,2.757134,0.036245,0.717319,1944040.500,48980780.0,10.581313,1449.598877,30.653315,1.0,39.041889,...,2007.004883,4.847284,2.546997,0.086840,0.0,2001.590454,9.169843,1.255913,5.089680,0.0
4,2.892810,0.112710,0.942127,2687474.500,46951980.0,6.817623,1686.934692,30.064833,1.0,48.286400,...,2007.097778,2.822258,2.856855,0.147626,0.0,2001.197632,10.504032,2.072754,5.173223,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,2.888023,0.062809,0.922750,1468823.625,51978432.0,1.882124,1674.410889,29.836220,1.0,27.595964,...,2006.653320,1.744773,3.436805,0.093086,0.0,2000.531128,11.134349,1.560324,5.503013,0.0
3996,2.678097,0.081119,0.941434,2207495.250,47915148.0,4.367768,2094.287598,33.201077,1.0,23.761257,...,2006.617432,3.135401,2.208577,0.138845,0.0,2000.823975,10.434250,2.738748,5.503053,0.0
3997,2.818056,0.071073,0.913206,2378230.000,51649904.0,5.824863,2925.216309,31.385662,1.0,64.372284,...,2005.998413,5.039365,4.110182,0.213476,0.0,2002.091797,10.432568,2.420090,5.380797,0.0
3998,2.941599,0.016656,0.964354,2471169.750,51876460.0,3.982392,2235.678467,30.020813,1.0,37.190655,...,2005.418701,2.873089,3.448856,0.157919,0.0,2001.137329,10.567339,3.008616,5.571043,0.0


In [477]:
generated_data["opioid_abuse"]

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
3995    1.0
3996    1.0
3997    1.0
3998    1.0
3999    1.0
Name: opioid_abuse, Length: 4000, dtype: float32

In [478]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10397 entries, 0 to 10396
Data columns (total 69 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   row_id_x              10397 non-null  int64         
 1   subject_id            10397 non-null  int64         
 2   hadm_id               10397 non-null  int64         
 3   admittime             10397 non-null  datetime64[ns]
 4   dischtime             10397 non-null  datetime64[ns]
 5   admission_type        10397 non-null  int64         
 6   hospital_expire_flag  10397 non-null  int64         
 7   has_chartevents_data  10397 non-null  int64         
 8   row_id_y              10397 non-null  float64       
 9   startdate             10397 non-null  datetime64[ns]
 10  enddate               10397 non-null  datetime64[ns]
 11  dose_val_rx           10397 non-null  float64       
 12  dose_unit_rx          10397 non-null  int64         
 13  form_val_disp   

In [479]:
grouped=df.groupby("subject_id")

In [480]:
generated_data

,admission_type,hospital_expire_flag,has_chartevents_data,row_id_y,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,opioid_abuse,admdiff,...,startdateyear,startdatemonth,startdateday,startdateday_of_week,startdatehour,enddateyear,enddatemonth,enddateday,enddateday_of_week,enddatehour
0,2.619910,0.071423,0.965048,2514856.000,51321884.0,4.272938,2171.296631,30.776081,1.0,35.295570,...,2008.258179,2.550713,1.780574,0.124380,0.0,2000.946899,10.948043,2.368855,5.570255,0.0
1,2.835562,0.036966,0.922353,2230993.500,49811132.0,6.087144,1454.517456,32.827713,1.0,33.724800,...,2007.290527,3.080406,1.844749,0.168674,0.0,2000.933105,10.492146,2.314125,5.261910,0.0
2,2.907156,0.025290,0.979776,3246302.250,51741772.0,2.822199,555.575806,33.118919,1.0,32.817257,...,2007.177734,3.623435,1.530835,0.015057,0.0,2000.611206,10.907797,1.276327,5.095401,0.0
3,2.757134,0.036245,0.717319,1944040.500,48980780.0,10.581313,1449.598877,30.653315,1.0,39.041889,...,2007.004883,4.847284,2.546997,0.086840,0.0,2001.590454,9.169843,1.255913,5.089680,0.0
4,2.892810,0.112710,0.942127,2687474.500,46951980.0,6.817623,1686.934692,30.064833,1.0,48.286400,...,2007.097778,2.822258,2.856855,0.147626,0.0,2001.197632,10.504032,2.072754,5.173223,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,2.888023,0.062809,0.922750,1468823.625,51978432.0,1.882124,1674.410889,29.836220,1.0,27.595964,...,2006.653320,1.744773,3.436805,0.093086,0.0,2000.531128,11.134349,1.560324,5.503013,0.0
3996,2.678097,0.081119,0.941434,2207495.250,47915148.0,4.367768,2094.287598,33.201077,1.0,23.761257,...,2006.617432,3.135401,2.208577,0.138845,0.0,2000.823975,10.434250,2.738748,5.503053,0.0
3997,2.818056,0.071073,0.913206,2378230.000,51649904.0,5.824863,2925.216309,31.385662,1.0,64.372284,...,2005.998413,5.039365,4.110182,0.213476,0.0,2002.091797,10.432568,2.420090,5.380797,0.0
3998,2.941599,0.016656,0.964354,2471169.750,51876460.0,3.982392,2235.678467,30.020813,1.0,37.190655,...,2005.418701,2.873089,3.448856,0.157919,0.0,2001.137329,10.567339,3.008616,5.571043,0.0


In [481]:
df["admittime"]

0       2000-01-14 18:42:00
1       2000-01-14 18:42:00
2       2000-01-14 18:42:00
3       2000-01-14 18:42:00
4       2000-01-14 18:42:00
                ...        
10392   2009-11-24 00:31:00
10393   2009-11-24 00:31:00
10394   2009-11-24 00:31:00
10395   2009-11-24 00:31:00
10396   2009-11-24 00:31:00
Name: admittime, Length: 10397, dtype: datetime64[ns]

In [482]:
df["admittimeyear"]

0        2000
1        2000
2        2000
3        2000
4        2000
         ... 
10392    2009
10393    2009
10394    2009
10395    2009
10396    2009
Name: admittimeyear, Length: 10397, dtype: int64

In [483]:
a=df["admittimeyear"].astype(int).astype(str)+"-"+df["admittimemonth"].astype(int).astype(str)+"-"+df["admittimeday"].astype(int).astype(str)+" "+df["admittimehour"].astype(int).astype(str)+":09:00"

In [484]:
a

0        2000-1-14 18:09:00
1        2000-1-14 18:09:00
2        2000-1-14 18:09:00
3        2000-1-14 18:09:00
4        2000-1-14 18:09:00
                ...        
10392    2009-11-24 0:09:00
10393    2009-11-24 0:09:00
10394    2009-11-24 0:09:00
10395    2009-11-24 0:09:00
10396    2009-11-24 0:09:00
Length: 10397, dtype: object

In [485]:
len(grouped.groups.keys())

94

In [486]:
s=0;e=42
c=max(df["hadm_id"])+1
for i in grouped.groups.keys():
    print(i)
    group=grouped.get_group(i)
    updt_group=pd.concat([group,generated_data[s:e]],ignore_index=True)
    updt_group["subject_id"]=[i for j in range(len(updt_group))]
    updt_group["hadm_id"]=[c for j in range(c,c+len(updt_group))]
    updt_group["admittime"]=updt_group["admittimeyear"].astype(int).astype(str)+"-"+updt_group["admittimemonth"].astype(int).astype(str)+"-"+updt_group["admittimeday"].astype(int).astype(str)+" "+updt_group["admittimehour"].astype(int).astype(str)+":09:00"
    updt_group["admittime"]=pd.to_datetime(updt_group["admittime"],errors="coerce")
    updt_group["dischtime"]=updt_group["admittimeyear"].astype(int).astype(str)+"-"+updt_group["admittimemonth"].astype(int).astype(str)+"-"+updt_group["admittimeday"].astype(int).astype(str)+" "+updt_group["admittimehour"].astype(int).astype(str)+":09:00"
    updt_group["dischtime"]=pd.to_datetime(updt_group["admittime"],errors="coerce")
    updt_group["startdate"]=updt_group["startdateyear"].astype(int).astype(str)+"-"+updt_group["startdatemonth"].astype(int).astype(str)+"-"+updt_group["startdateday"].astype(int).astype(str)+" "+updt_group["startdatehour"].astype(int).astype(str)+":09:00"
    updt_group["startdate"]=pd.to_datetime(updt_group["startdate"],errors="coerce")
    updt_group["enddate"]=updt_group["enddateyear"].astype(int).astype(str)+"-"+updt_group["enddatemonth"].astype(int).astype(str)+"-"+updt_group["enddateday"].astype(int).astype(str)+" "+updt_group["enddatehour"].astype(int).astype(str)+":09:00"
    updt_group["enddate"]=pd.to_datetime(updt_group["enddate"],errors="coerce")
    s+=42;e+=42;c+=len(updt_group)
    df_without_old_group = df[df['subject_id'] !=i]
    df= pd.concat([df_without_old_group, updt_group], ignore_index=True)
    

10006
10013
10017
10019
10026
10027
10029
10032
10033
10035
10036
10038
10040
10042
10043
10044
10045
10056
10059
10061
10064
10065
10074
10076
10083
10088
10089
10090
10093
10094
10098
10101
10102
10104
10106
10111
10112
10117
10119
10120
10124
10126
10127
10132
40124
40177
40204
40277
40286
40304
40310
40456
40503
40595
40601
40612
40655
40687
41795
41914
41976
41983
42033
42066
42075
42135
42199
42231
42275
42281
42292
42302
42321
42346
42367
42412
42430
42458
43735
43746
43748
43779
43798
43827
43870
43879
43881
43909
43927
44083
44154
44212
44222
44228


In [487]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14345 entries, 0 to 14344
Data columns (total 69 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   row_id_x              10397 non-null  float64       
 1   subject_id            14345 non-null  int64         
 2   hadm_id               14345 non-null  int64         
 3   admittime             14345 non-null  datetime64[ns]
 4   dischtime             14345 non-null  datetime64[ns]
 5   admission_type        14345 non-null  float64       
 6   hospital_expire_flag  14345 non-null  float64       
 7   has_chartevents_data  14345 non-null  float64       
 8   row_id_y              14345 non-null  float64       
 9   startdate             14345 non-null  datetime64[ns]
 10  enddate               14345 non-null  datetime64[ns]
 11  dose_val_rx           14345 non-null  float64       
 12  dose_unit_rx          14345 non-null  float64       
 13  form_val_disp   

In [488]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14345 entries, 0 to 14344
Data columns (total 69 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   row_id_x              10397 non-null  float64       
 1   subject_id            14345 non-null  int64         
 2   hadm_id               14345 non-null  int64         
 3   admittime             14345 non-null  datetime64[ns]
 4   dischtime             14345 non-null  datetime64[ns]
 5   admission_type        14345 non-null  float64       
 6   hospital_expire_flag  14345 non-null  float64       
 7   has_chartevents_data  14345 non-null  float64       
 8   row_id_y              14345 non-null  float64       
 9   startdate             14345 non-null  datetime64[ns]
 10  enddate               14345 non-null  datetime64[ns]
 11  dose_val_rx           14345 non-null  float64       
 12  dose_unit_rx          14345 non-null  float64       
 13  form_val_disp   

In [489]:
grouped.get_group(10089)

,row_id_x,subject_id,hadm_id,admittime,dischtime,admission_type,hospital_expire_flag,has_chartevents_data,row_id_y,startdate,...,startdateyear,startdatemonth,startdateday,startdateday_of_week,startdatehour,enddateyear,enddatemonth,enddateday,enddateday_of_week,enddatehour
2998,12340,10089,190301,2002-08-05 18:48:00,2002-08-08 02:15:00,1,1,1,2559946.0,2002-08-06,...,2002,8,6,1,0,2002,8,7,2,0
2999,12340,10089,190301,2002-08-05 18:48:00,2002-08-08 02:15:00,1,1,1,2559947.0,2002-08-06,...,2002,8,6,1,0,2002,8,7,2,0
3000,12340,10089,190301,2002-08-05 18:48:00,2002-08-08 02:15:00,1,1,1,2559948.0,2002-08-06,...,2002,8,6,1,0,2002,8,7,2,0
3001,12340,10089,190301,2002-08-05 18:48:00,2002-08-08 02:15:00,1,1,1,2559949.0,2002-08-06,...,2002,8,6,1,0,2002,8,7,2,0
3002,12340,10089,190301,2002-08-05 18:48:00,2002-08-08 02:15:00,1,1,1,2559954.0,2002-08-07,...,2002,8,7,2,0,2002,8,7,2,0
3003,12340,10089,190301,2002-08-05 18:48:00,2002-08-08 02:15:00,1,1,1,2559951.0,2002-08-06,...,2002,8,6,1,0,2002,8,7,2,0
3004,12340,10089,190301,2002-08-05 18:48:00,2002-08-08 02:15:00,1,1,1,2559952.0,2002-08-06,...,2002,8,6,1,0,2002,8,7,2,0
3005,12340,10089,190301,2002-08-05 18:48:00,2002-08-08 02:15:00,1,1,1,2559953.0,2002-08-06,...,2002,8,6,1,0,2002,8,7,2,0
3006,12340,10089,190301,2002-08-05 18:48:00,2002-08-08 02:15:00,1,1,1,2559955.0,2002-08-07,...,2002,8,7,2,0,2002,8,7,2,0
3007,12340,10089,190301,2002-08-05 18:48:00,2002-08-08 02:15:00,1,1,1,2559956.0,2002-08-07,...,2002,8,7,2,0,2002,8,8,3,0


In [490]:
df

,row_id_x,subject_id,hadm_id,admittime,dischtime,admission_type,hospital_expire_flag,has_chartevents_data,row_id_y,startdate,...,startdateyear,startdatemonth,startdateday,startdateday_of_week,startdatehour,enddateyear,enddatemonth,enddateday,enddateday_of_week,enddatehour
0,12258.0,10006,199396,2004-10-23 21:09:00,2004-10-23 21:09:00,1.000000,0.000000,1.000000,299263.00,2004-10-23 00:09:00,...,2004.000000,10.000000,23.000000,5.000000,0.0,2004.000000,10.000000,23.000000,5.000000,0.0
1,12258.0,10006,199396,2004-10-23 21:09:00,2004-10-23 21:09:00,1.000000,0.000000,1.000000,299262.00,2004-10-23 00:09:00,...,2004.000000,10.000000,23.000000,5.000000,0.0,2004.000000,10.000000,24.000000,6.000000,0.0
2,12258.0,10006,199396,2004-10-23 21:09:00,2004-10-23 21:09:00,1.000000,0.000000,1.000000,299261.00,2004-10-23 00:09:00,...,2004.000000,10.000000,23.000000,5.000000,0.0,2004.000000,10.000000,24.000000,6.000000,0.0
3,12258.0,10006,199396,2004-10-23 21:09:00,2004-10-23 21:09:00,1.000000,0.000000,1.000000,299257.00,2004-11-01 00:09:00,...,2004.000000,11.000000,1.000000,0.000000,0.0,2004.000000,11.000000,1.000000,0.000000,0.0
4,12258.0,10006,199396,2004-10-23 21:09:00,2004-10-23 21:09:00,1.000000,0.000000,1.000000,299259.00,2004-11-01 00:09:00,...,2004.000000,11.000000,1.000000,0.000000,0.0,2004.000000,11.000000,1.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14340,NaN,44228,213603,2001-02-13 12:09:00,2001-02-13 12:09:00,2.770105,0.141276,0.872150,2791178.75,2005-03-02 00:09:00,...,2005.793823,3.074370,2.336696,0.128959,0.0,2000.884521,9.335856,2.238024,4.995748,0.0
14341,NaN,44228,213603,2000-02-23 07:09:00,2000-02-23 07:09:00,2.856957,0.045161,0.949577,2324464.75,2004-02-02 00:09:00,...,2004.427490,2.473697,2.455400,0.095495,0.0,2001.187012,10.243123,1.813247,5.358908,0.0
14342,NaN,44228,213603,2000-04-17 05:09:00,2000-04-17 05:09:00,2.878041,0.073683,0.910976,2215265.00,2006-02-01 00:09:00,...,2006.905273,2.501574,1.874562,0.156227,0.0,2001.567383,10.482417,2.198515,5.119682,0.0
14343,NaN,44228,213603,2000-02-22 09:09:00,2000-02-22 09:09:00,2.835004,0.055065,0.892200,2491089.50,2005-03-01 00:09:00,...,2005.731079,3.756621,1.379601,0.116255,0.0,2001.544922,11.145884,2.202842,5.486466,0.0


In [491]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14345 entries, 0 to 14344
Data columns (total 69 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   row_id_x              10397 non-null  float64       
 1   subject_id            14345 non-null  int64         
 2   hadm_id               14345 non-null  int64         
 3   admittime             14345 non-null  datetime64[ns]
 4   dischtime             14345 non-null  datetime64[ns]
 5   admission_type        14345 non-null  float64       
 6   hospital_expire_flag  14345 non-null  float64       
 7   has_chartevents_data  14345 non-null  float64       
 8   row_id_y              14345 non-null  float64       
 9   startdate             14345 non-null  datetime64[ns]
 10  enddate               14345 non-null  datetime64[ns]
 11  dose_val_rx           14345 non-null  float64       
 12  dose_unit_rx          14345 non-null  float64       
 13  form_val_disp   

In [492]:
df=df.dropna(subset=["enddate"],axis=0)

In [493]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14345 entries, 0 to 14344
Data columns (total 69 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   row_id_x              10397 non-null  float64       
 1   subject_id            14345 non-null  int64         
 2   hadm_id               14345 non-null  int64         
 3   admittime             14345 non-null  datetime64[ns]
 4   dischtime             14345 non-null  datetime64[ns]
 5   admission_type        14345 non-null  float64       
 6   hospital_expire_flag  14345 non-null  float64       
 7   has_chartevents_data  14345 non-null  float64       
 8   row_id_y              14345 non-null  float64       
 9   startdate             14345 non-null  datetime64[ns]
 10  enddate               14345 non-null  datetime64[ns]
 11  dose_val_rx           14345 non-null  float64       
 12  dose_unit_rx          14345 non-null  float64       
 13  form_val_disp   

In [494]:
new_cols_names={'admlV': 'admission_location',
 'dischlV': 'discharge_location',
 'insV': 'insurance',
 'mrgsV': 'marital_status',
 'diagV': 'diagnosis',
 'drgtV': 'drug_type',
 'drgV': 'drug',
 'drgnpV': 'drug_name_poe',
 'drgngV': 'drug_name_generic',
 'routeV': 'route'}

In [495]:
df

,row_id_x,subject_id,hadm_id,admittime,dischtime,admission_type,hospital_expire_flag,has_chartevents_data,row_id_y,startdate,...,startdateyear,startdatemonth,startdateday,startdateday_of_week,startdatehour,enddateyear,enddatemonth,enddateday,enddateday_of_week,enddatehour
0,12258.0,10006,199396,2004-10-23 21:09:00,2004-10-23 21:09:00,1.000000,0.000000,1.000000,299263.00,2004-10-23 00:09:00,...,2004.000000,10.000000,23.000000,5.000000,0.0,2004.000000,10.000000,23.000000,5.000000,0.0
1,12258.0,10006,199396,2004-10-23 21:09:00,2004-10-23 21:09:00,1.000000,0.000000,1.000000,299262.00,2004-10-23 00:09:00,...,2004.000000,10.000000,23.000000,5.000000,0.0,2004.000000,10.000000,24.000000,6.000000,0.0
2,12258.0,10006,199396,2004-10-23 21:09:00,2004-10-23 21:09:00,1.000000,0.000000,1.000000,299261.00,2004-10-23 00:09:00,...,2004.000000,10.000000,23.000000,5.000000,0.0,2004.000000,10.000000,24.000000,6.000000,0.0
3,12258.0,10006,199396,2004-10-23 21:09:00,2004-10-23 21:09:00,1.000000,0.000000,1.000000,299257.00,2004-11-01 00:09:00,...,2004.000000,11.000000,1.000000,0.000000,0.0,2004.000000,11.000000,1.000000,0.000000,0.0
4,12258.0,10006,199396,2004-10-23 21:09:00,2004-10-23 21:09:00,1.000000,0.000000,1.000000,299259.00,2004-11-01 00:09:00,...,2004.000000,11.000000,1.000000,0.000000,0.0,2004.000000,11.000000,1.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14340,NaN,44228,213603,2001-02-13 12:09:00,2001-02-13 12:09:00,2.770105,0.141276,0.872150,2791178.75,2005-03-02 00:09:00,...,2005.793823,3.074370,2.336696,0.128959,0.0,2000.884521,9.335856,2.238024,4.995748,0.0
14341,NaN,44228,213603,2000-02-23 07:09:00,2000-02-23 07:09:00,2.856957,0.045161,0.949577,2324464.75,2004-02-02 00:09:00,...,2004.427490,2.473697,2.455400,0.095495,0.0,2001.187012,10.243123,1.813247,5.358908,0.0
14342,NaN,44228,213603,2000-04-17 05:09:00,2000-04-17 05:09:00,2.878041,0.073683,0.910976,2215265.00,2006-02-01 00:09:00,...,2006.905273,2.501574,1.874562,0.156227,0.0,2001.567383,10.482417,2.198515,5.119682,0.0
14343,NaN,44228,213603,2000-02-22 09:09:00,2000-02-22 09:09:00,2.835004,0.055065,0.892200,2491089.50,2005-03-01 00:09:00,...,2005.731079,3.756621,1.379601,0.116255,0.0,2001.544922,11.145884,2.202842,5.486466,0.0


In [496]:
df=df.drop("row_id_x",axis=1)

In [497]:
df

,subject_id,hadm_id,admittime,dischtime,admission_type,hospital_expire_flag,has_chartevents_data,row_id_y,startdate,enddate,...,startdateyear,startdatemonth,startdateday,startdateday_of_week,startdatehour,enddateyear,enddatemonth,enddateday,enddateday_of_week,enddatehour
0,10006,199396,2004-10-23 21:09:00,2004-10-23 21:09:00,1.000000,0.000000,1.000000,299263.00,2004-10-23 00:09:00,2004-10-23 00:09:00,...,2004.000000,10.000000,23.000000,5.000000,0.0,2004.000000,10.000000,23.000000,5.000000,0.0
1,10006,199396,2004-10-23 21:09:00,2004-10-23 21:09:00,1.000000,0.000000,1.000000,299262.00,2004-10-23 00:09:00,2004-10-24 00:09:00,...,2004.000000,10.000000,23.000000,5.000000,0.0,2004.000000,10.000000,24.000000,6.000000,0.0
2,10006,199396,2004-10-23 21:09:00,2004-10-23 21:09:00,1.000000,0.000000,1.000000,299261.00,2004-10-23 00:09:00,2004-10-24 00:09:00,...,2004.000000,10.000000,23.000000,5.000000,0.0,2004.000000,10.000000,24.000000,6.000000,0.0
3,10006,199396,2004-10-23 21:09:00,2004-10-23 21:09:00,1.000000,0.000000,1.000000,299257.00,2004-11-01 00:09:00,2004-11-01 00:09:00,...,2004.000000,11.000000,1.000000,0.000000,0.0,2004.000000,11.000000,1.000000,0.000000,0.0
4,10006,199396,2004-10-23 21:09:00,2004-10-23 21:09:00,1.000000,0.000000,1.000000,299259.00,2004-11-01 00:09:00,2004-11-01 00:09:00,...,2004.000000,11.000000,1.000000,0.000000,0.0,2004.000000,11.000000,1.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14340,44228,213603,2001-02-13 12:09:00,2001-02-13 12:09:00,2.770105,0.141276,0.872150,2791178.75,2005-03-02 00:09:00,2000-09-02 00:09:00,...,2005.793823,3.074370,2.336696,0.128959,0.0,2000.884521,9.335856,2.238024,4.995748,0.0
14341,44228,213603,2000-02-23 07:09:00,2000-02-23 07:09:00,2.856957,0.045161,0.949577,2324464.75,2004-02-02 00:09:00,2001-10-01 00:09:00,...,2004.427490,2.473697,2.455400,0.095495,0.0,2001.187012,10.243123,1.813247,5.358908,0.0
14342,44228,213603,2000-04-17 05:09:00,2000-04-17 05:09:00,2.878041,0.073683,0.910976,2215265.00,2006-02-01 00:09:00,2001-10-02 00:09:00,...,2006.905273,2.501574,1.874562,0.156227,0.0,2001.567383,10.482417,2.198515,5.119682,0.0
14343,44228,213603,2000-02-22 09:09:00,2000-02-22 09:09:00,2.835004,0.055065,0.892200,2491089.50,2005-03-01 00:09:00,2001-11-02 00:09:00,...,2005.731079,3.756621,1.379601,0.116255,0.0,2001.544922,11.145884,2.202842,5.486466,0.0


In [498]:
from sklearn.decomposition import PCA

In [499]:
pca=PCA(n_components=20)

In [500]:
cols=df.columns

In [501]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14345 entries, 0 to 14344
Data columns (total 68 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   subject_id            14345 non-null  int64         
 1   hadm_id               14345 non-null  int64         
 2   admittime             14345 non-null  datetime64[ns]
 3   dischtime             14345 non-null  datetime64[ns]
 4   admission_type        14345 non-null  float64       
 5   hospital_expire_flag  14345 non-null  float64       
 6   has_chartevents_data  14345 non-null  float64       
 7   row_id_y              14345 non-null  float64       
 8   startdate             14345 non-null  datetime64[ns]
 9   enddate               14345 non-null  datetime64[ns]
 10  dose_val_rx           14345 non-null  float64       
 11  dose_unit_rx          14345 non-null  float64       
 12  form_val_disp         14345 non-null  float64       
 13  form_unit_disp  

In [502]:
df=df.drop([cols[2],cols[3],cols[8],cols[9]],axis=1)

In [503]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14345 entries, 0 to 14344
Data columns (total 64 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   subject_id            14345 non-null  int64  
 1   hadm_id               14345 non-null  int64  
 2   admission_type        14345 non-null  float64
 3   hospital_expire_flag  14345 non-null  float64
 4   has_chartevents_data  14345 non-null  float64
 5   row_id_y              14345 non-null  float64
 6   dose_val_rx           14345 non-null  float64
 7   dose_unit_rx          14345 non-null  float64
 8   form_val_disp         14345 non-null  float64
 9   form_unit_disp        14345 non-null  float64
 10  opioid_abuse          14345 non-null  float64
 11  admdiff               14345 non-null  float64
 12  dosgdiff              14345 non-null  float64
 13  maxdays               14345 non-null  float64
 14  admlV1                14345 non-null  float64
 15  admlV2             

In [504]:
df

,subject_id,hadm_id,admission_type,hospital_expire_flag,has_chartevents_data,row_id_y,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,...,startdateyear,startdatemonth,startdateday,startdateday_of_week,startdatehour,enddateyear,enddatemonth,enddateday,enddateday_of_week,enddatehour
0,10006,199396,1.000000,0.000000,1.000000,299263.00,1.0,7.000000,0.500000,27.000000,...,2004.000000,10.000000,23.000000,5.000000,0.0,2004.000000,10.000000,23.000000,5.000000,0.0
1,10006,199396,1.000000,0.000000,1.000000,299262.00,1000.0,4.000000,1000.000000,4.000000,...,2004.000000,10.000000,23.000000,5.000000,0.0,2004.000000,10.000000,24.000000,6.000000,0.0
2,10006,199396,1.000000,0.000000,1.000000,299261.00,500.0,4.000000,1.000000,14.000000,...,2004.000000,10.000000,23.000000,5.000000,0.0,2004.000000,10.000000,24.000000,6.000000,0.0
3,10006,199396,1.000000,0.000000,1.000000,299257.00,500.0,4.000000,1.000000,17.000000,...,2004.000000,11.000000,1.000000,0.000000,0.0,2004.000000,11.000000,1.000000,0.000000,0.0
4,10006,199396,1.000000,0.000000,1.000000,299259.00,250.0,4.000000,1.000000,17.000000,...,2004.000000,11.000000,1.000000,0.000000,0.0,2004.000000,11.000000,1.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14340,44228,213603,2.770105,0.141276,0.872150,2791178.75,51547664.0,3.859945,1491.207397,31.737547,...,2005.793823,3.074370,2.336696,0.128959,0.0,2000.884521,9.335856,2.238024,4.995748,0.0
14341,44228,213603,2.856957,0.045161,0.949577,2324464.75,51472036.0,4.630011,1673.760620,31.357889,...,2004.427490,2.473697,2.455400,0.095495,0.0,2001.187012,10.243123,1.813247,5.358908,0.0
14342,44228,213603,2.878041,0.073683,0.910976,2215265.00,50200148.0,9.979579,1170.548096,31.231541,...,2006.905273,2.501574,1.874562,0.156227,0.0,2001.567383,10.482417,2.198515,5.119682,0.0
14343,44228,213603,2.835004,0.055065,0.892200,2491089.50,50602412.0,5.958428,1168.348755,30.362192,...,2005.731079,3.756621,1.379601,0.116255,0.0,2001.544922,11.145884,2.202842,5.486466,0.0


In [505]:
df=df.drop("row_id_x",axis=1)

KeyError: "['row_id_x'] not found in axis"

In [506]:
principal_components = pca.fit_transform(df)

In [507]:
principal_components[0]

array([-1.38030882e+07,  2.01356890e+06, -4.19960829e+03, -7.72301632e+03,
        2.92195054e+03, -5.44261277e+01, -3.19492015e-01,  6.39765129e-01,
       -1.12338538e+01,  1.54521815e+01,  7.40064483e+00,  8.97332843e-01,
        8.61034213e+00,  8.82649043e+00, -4.33822719e+00, -7.64350769e+00,
       -5.29831593e+00,  5.30489451e-01, -2.87742993e+00,  3.30387084e+00])

In [508]:
print(df.var().sort_values(ascending=False)[:60])

dose_val_rx             5.008886e+14
row_id_y                9.509520e+11
maxdays                 1.686447e+09
subject_id              2.508150e+08
hadm_id                 1.695602e+07
form_val_disp           2.904214e+06
dosgdiff                8.294670e+04
admdiff                 8.067625e+02
form_unit_disp          9.757564e+01
enddateday              9.208031e+01
startdateday            8.683128e+01
dischtimeday            7.916706e+01
admittimeday            6.272202e+01
dischtimehour           4.572680e+01
admittimehour           3.394411e+01
dose_unit_rx            2.880472e+01
startdatemonth          1.278374e+01
admittimemonth          1.216160e+01
enddatemonth            1.135792e+01
dischtimemonth          1.083364e+01
admittimeyear           9.456954e+00
enddateyear             8.469484e+00
startdateyear           7.973668e+00
dischtimeyear           7.624239e+00
startdateday_of_week    4.343321e+00
enddateday_of_week      3.536454e+00
admittimeday_of_week    3.133592e+00
d

In [509]:
a

0        2000-1-14 18:09:00
1        2000-1-14 18:09:00
2        2000-1-14 18:09:00
3        2000-1-14 18:09:00
4        2000-1-14 18:09:00
                ...        
10392    2009-11-24 0:09:00
10393    2009-11-24 0:09:00
10394    2009-11-24 0:09:00
10395    2009-11-24 0:09:00
10396    2009-11-24 0:09:00
Length: 10397, dtype: object

In [510]:
sorted(a)

['2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14 18:09:00',
 '2000-1-14

In [511]:
df.describe()

,subject_id,hadm_id,admission_type,hospital_expire_flag,has_chartevents_data,row_id_y,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,...,startdateyear,startdatemonth,startdateday,startdateday_of_week,startdatehour,enddateyear,enddatemonth,enddateday,enddateday_of_week,enddatehour
count,14345.000000,14345.000000,14345.000000,14345.000000,14345.000000,1.434500e+04,1.434500e+04,14345.000000,14345.000000,14345.000000,...,14345.000000,14345.000000,14345.000000,14345.000000,14345.0,14345.000000,14345.000000,14345.000000,14345.000000,14345.0
mean,28461.084908,206412.718857,1.558565,0.274973,0.975577,2.358474e+06,1.379630e+07,7.415203,629.378752,20.285552,...,2005.073442,5.657909,11.985000,2.216975,0.0,2003.475488,7.879024,11.881513,3.699792,0.0
std,15837.139341,4117.769109,0.821911,0.420729,0.086428,9.751677e+05,2.238054e+07,5.367003,1704.175344,9.878038,...,2.823768,3.575435,9.318330,2.084064,0.0,2.910238,3.370151,9.595849,1.880546,0.0
min,10006.000000,199396.000000,1.000000,0.000000,0.000000,3.260000e+04,0.000000e+00,1.000000,0.000000,1.000000,...,2000.000000,1.000000,1.000000,0.000000,0.0,2000.000000,1.000000,1.000000,0.000000,0.0
25%,10093.000000,202905.000000,1.000000,0.000000,0.975800,1.574705e+06,1.500000e+01,4.000000,1.000000,14.000000,...,2002.000000,2.209290,3.021729,0.121882,0.0,2001.000000,5.000000,2.307564,2.000000,0.0
50%,40310.000000,206164.000000,1.000000,0.039962,1.000000,2.562746e+06,2.000000e+02,5.644795,2.000000,21.000000,...,2005.917603,5.000000,10.000000,2.000000,0.0,2002.000000,9.000000,10.000000,4.000000,0.0
75%,42135.000000,209978.000000,2.632957,0.276092,1.000000,3.308565e+06,4.758876e+07,9.088881,1061.778564,28.897509,...,2007.260254,9.000000,19.000000,4.000000,0.0,2006.000000,10.584756,19.000000,5.162497,0.0
max,44228.000000,213603.000000,3.000000,1.000000,1.000000,3.609917e+06,5.250750e+07,35.000000,100500.000000,37.000000,...,2009.000000,12.000000,31.000000,6.000000,0.0,2009.000000,12.000000,31.000000,6.000000,0.0


In [512]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14345 entries, 0 to 14344
Data columns (total 64 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   subject_id            14345 non-null  int64  
 1   hadm_id               14345 non-null  int64  
 2   admission_type        14345 non-null  float64
 3   hospital_expire_flag  14345 non-null  float64
 4   has_chartevents_data  14345 non-null  float64
 5   row_id_y              14345 non-null  float64
 6   dose_val_rx           14345 non-null  float64
 7   dose_unit_rx          14345 non-null  float64
 8   form_val_disp         14345 non-null  float64
 9   form_unit_disp        14345 non-null  float64
 10  opioid_abuse          14345 non-null  float64
 11  admdiff               14345 non-null  float64
 12  dosgdiff              14345 non-null  float64
 13  maxdays               14345 non-null  float64
 14  admlV1                14345 non-null  float64
 15  admlV2             

In [513]:
cols=df.columns

In [514]:
df=df.drop(cols[14:20],axis=1)

In [515]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14345 entries, 0 to 14344
Data columns (total 58 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   subject_id            14345 non-null  int64  
 1   hadm_id               14345 non-null  int64  
 2   admission_type        14345 non-null  float64
 3   hospital_expire_flag  14345 non-null  float64
 4   has_chartevents_data  14345 non-null  float64
 5   row_id_y              14345 non-null  float64
 6   dose_val_rx           14345 non-null  float64
 7   dose_unit_rx          14345 non-null  float64
 8   form_val_disp         14345 non-null  float64
 9   form_unit_disp        14345 non-null  float64
 10  opioid_abuse          14345 non-null  float64
 11  admdiff               14345 non-null  float64
 12  dosgdiff              14345 non-null  float64
 13  maxdays               14345 non-null  float64
 14  insV1                 14345 non-null  float64
 15  insV2              

In [516]:
pd.read_csv("merged_df.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'merged_df.csv'

In [ ]:
df["admittimeyear"]=df["admittimeyear"]+(df["admittimemonth"]/12)+(df["admittimeday"]/(30*12))+(df["admittimehour"]/(24*30*12))

In [517]:
df["dischtimeyear"]=df["dischtimeyear"]+(df["dischtimemonth"]/12)+(df["dischtimeday"]/(30*12))+(df["dischtimehour"]/(24*30*12))

In [518]:
df["startdateyear"]=df["startdateyear"]+(df["startdatemonth"]/12)+(df["startdateday"]/(30*12))+(df["startdatehour"]/(24*30*12))

In [519]:
df["enddateyear"]=df["enddateyear"]+(df["enddatemonth"]/12)+(df["enddateday"]/(30*12))+(df["enddatehour"]/(24*30*12))

In [520]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14345 entries, 0 to 14344
Data columns (total 58 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   subject_id            14345 non-null  int64  
 1   hadm_id               14345 non-null  int64  
 2   admission_type        14345 non-null  float64
 3   hospital_expire_flag  14345 non-null  float64
 4   has_chartevents_data  14345 non-null  float64
 5   row_id_y              14345 non-null  float64
 6   dose_val_rx           14345 non-null  float64
 7   dose_unit_rx          14345 non-null  float64
 8   form_val_disp         14345 non-null  float64
 9   form_unit_disp        14345 non-null  float64
 10  opioid_abuse          14345 non-null  float64
 11  admdiff               14345 non-null  float64
 12  dosgdiff              14345 non-null  float64
 13  maxdays               14345 non-null  float64
 14  insV1                 14345 non-null  float64
 15  insV2              

In [521]:
cols=df.columns

In [522]:
cols

Index(['subject_id', 'hadm_id', 'admission_type', 'hospital_expire_flag',
       'has_chartevents_data', 'row_id_y', 'dose_val_rx', 'dose_unit_rx',
       'form_val_disp', 'form_unit_disp', 'opioid_abuse', 'admdiff',
       'dosgdiff', 'maxdays', 'insV1', 'insV2', 'insV3', 'mrgsV1', 'mrgsV2',
       'mrgsV3', 'diagV1', 'diagV2', 'diagV3', 'drgtV1', 'drgtV2', 'drgtV3',
       'drgV1', 'drgV2', 'drgV3', 'drgnpV1', 'drgnpV2', 'drgnpV3', 'drgngV1',
       'drgngV2', 'drgngV3', 'routeV1', 'routeV2', 'routeV3', 'admittimeyear',
       'admittimemonth', 'admittimeday', 'admittimeday_of_week',
       'admittimehour', 'dischtimeyear', 'dischtimemonth', 'dischtimeday',
       'dischtimeday_of_week', 'dischtimehour', 'startdateyear',
       'startdatemonth', 'startdateday', 'startdateday_of_week',
       'startdatehour', 'enddateyear', 'enddatemonth', 'enddateday',
       'enddateday_of_week', 'enddatehour'],
      dtype='object')

In [523]:
df=df.drop([cols[39],cols[40],cols[42],cols[44],cols[45],cols[47],cols[49],cols[50],cols[52],cols[54],cols[55],cols[57]],axis=1)

In [524]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14345 entries, 0 to 14344
Data columns (total 46 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   subject_id            14345 non-null  int64  
 1   hadm_id               14345 non-null  int64  
 2   admission_type        14345 non-null  float64
 3   hospital_expire_flag  14345 non-null  float64
 4   has_chartevents_data  14345 non-null  float64
 5   row_id_y              14345 non-null  float64
 6   dose_val_rx           14345 non-null  float64
 7   dose_unit_rx          14345 non-null  float64
 8   form_val_disp         14345 non-null  float64
 9   form_unit_disp        14345 non-null  float64
 10  opioid_abuse          14345 non-null  float64
 11  admdiff               14345 non-null  float64
 12  dosgdiff              14345 non-null  float64
 13  maxdays               14345 non-null  float64
 14  insV1                 14345 non-null  float64
 15  insV2              

In [525]:
df[cols[4]]

0        1.000000
1        1.000000
2        1.000000
3        1.000000
4        1.000000
           ...   
14340    0.872150
14341    0.949577
14342    0.910976
14343    0.892200
14344    0.922996
Name: has_chartevents_data, Length: 14345, dtype: float64

In [526]:
df=df.drop(cols[35:38],axis=1)

In [527]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14345 entries, 0 to 14344
Data columns (total 43 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   subject_id            14345 non-null  int64  
 1   hadm_id               14345 non-null  int64  
 2   admission_type        14345 non-null  float64
 3   hospital_expire_flag  14345 non-null  float64
 4   has_chartevents_data  14345 non-null  float64
 5   row_id_y              14345 non-null  float64
 6   dose_val_rx           14345 non-null  float64
 7   dose_unit_rx          14345 non-null  float64
 8   form_val_disp         14345 non-null  float64
 9   form_unit_disp        14345 non-null  float64
 10  opioid_abuse          14345 non-null  float64
 11  admdiff               14345 non-null  float64
 12  dosgdiff              14345 non-null  float64
 13  maxdays               14345 non-null  float64
 14  insV1                 14345 non-null  float64
 15  insV2              

In [528]:
cols=df.columns

In [529]:
df=df.drop(cols[4:6],axis=1)

In [530]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14345 entries, 0 to 14344
Data columns (total 41 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   subject_id            14345 non-null  int64  
 1   hadm_id               14345 non-null  int64  
 2   admission_type        14345 non-null  float64
 3   hospital_expire_flag  14345 non-null  float64
 4   dose_val_rx           14345 non-null  float64
 5   dose_unit_rx          14345 non-null  float64
 6   form_val_disp         14345 non-null  float64
 7   form_unit_disp        14345 non-null  float64
 8   opioid_abuse          14345 non-null  float64
 9   admdiff               14345 non-null  float64
 10  dosgdiff              14345 non-null  float64
 11  maxdays               14345 non-null  float64
 12  insV1                 14345 non-null  float64
 13  insV2                 14345 non-null  float64
 14  insV3                 14345 non-null  float64
 15  mrgsV1             

In [531]:
df

,subject_id,hadm_id,admission_type,hospital_expire_flag,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,opioid_abuse,admdiff,...,drgngV2,drgngV3,admittimeyear,admittimeday_of_week,dischtimeyear,dischtimeday_of_week,startdateyear,startdateday_of_week,enddateyear,enddateday_of_week
0,10006,199396,1.000000,0.000000,1.0,7.000000,0.500000,27.000000,0.0,9.000000,...,0.501900,-0.617133,2004.000000,5.000000,2004.921412,0.000000,2004.897222,5.000000,2004.897222,5.000000
1,10006,199396,1.000000,0.000000,1000.0,4.000000,1000.000000,4.000000,0.0,9.000000,...,0.504449,-0.615771,2004.000000,5.000000,2004.921412,0.000000,2004.897222,5.000000,2004.900000,6.000000
2,10006,199396,1.000000,0.000000,500.0,4.000000,1.000000,14.000000,0.0,9.000000,...,0.553635,-0.688120,2004.000000,5.000000,2004.921412,0.000000,2004.897222,5.000000,2004.900000,6.000000
3,10006,199396,1.000000,0.000000,500.0,4.000000,1.000000,17.000000,0.0,9.000000,...,0.553635,-0.688120,2004.000000,5.000000,2004.921412,0.000000,2004.919444,0.000000,2004.919444,0.000000
4,10006,199396,1.000000,0.000000,250.0,4.000000,1.000000,17.000000,0.0,9.000000,...,0.560983,-0.683589,2004.000000,5.000000,2004.921412,0.000000,2004.919444,0.000000,2004.919444,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14340,44228,213603,2.770105,0.141276,51547664.0,3.859945,1491.207397,31.737547,1.0,38.700829,...,0.555630,-0.107540,2001.047363,1.259752,2007.029381,3.384760,2006.056512,0.128959,2001.668726,4.995748
14341,44228,213603,2.856957,0.045161,51472036.0,4.630011,1673.760620,31.357889,1.0,36.073734,...,0.559667,-0.119350,2000.607788,2.354616,2006.344867,4.565771,2004.640452,0.095495,2002.045642,5.358908
14342,44228,213603,2.878041,0.073683,50200148.0,9.979579,1170.548096,31.231541,1.0,28.988939,...,0.552917,-0.175002,2000.481689,2.840288,2006.955256,3.966846,2007.118945,0.156227,2002.447025,5.119682
14343,44228,213603,2.835004,0.055065,50602412.0,5.958428,1168.348755,30.362192,1.0,32.580879,...,0.556924,-0.126985,2000.767578,2.207397,2006.120409,2.420683,2006.047963,0.116255,2002.479865,5.486466


In [532]:
cols=df.columns

In [533]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14345 entries, 0 to 14344
Data columns (total 41 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   subject_id            14345 non-null  int64  
 1   hadm_id               14345 non-null  int64  
 2   admission_type        14345 non-null  float64
 3   hospital_expire_flag  14345 non-null  float64
 4   dose_val_rx           14345 non-null  float64
 5   dose_unit_rx          14345 non-null  float64
 6   form_val_disp         14345 non-null  float64
 7   form_unit_disp        14345 non-null  float64
 8   opioid_abuse          14345 non-null  float64
 9   admdiff               14345 non-null  float64
 10  dosgdiff              14345 non-null  float64
 11  maxdays               14345 non-null  float64
 12  insV1                 14345 non-null  float64
 13  insV2                 14345 non-null  float64
 14  insV3                 14345 non-null  float64
 15  mrgsV1             

In [534]:
df=df.drop([cols[34],cols[36],cols[38],cols[40]],axis=1)

In [535]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14345 entries, 0 to 14344
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   subject_id            14345 non-null  int64  
 1   hadm_id               14345 non-null  int64  
 2   admission_type        14345 non-null  float64
 3   hospital_expire_flag  14345 non-null  float64
 4   dose_val_rx           14345 non-null  float64
 5   dose_unit_rx          14345 non-null  float64
 6   form_val_disp         14345 non-null  float64
 7   form_unit_disp        14345 non-null  float64
 8   opioid_abuse          14345 non-null  float64
 9   admdiff               14345 non-null  float64
 10  dosgdiff              14345 non-null  float64
 11  maxdays               14345 non-null  float64
 12  insV1                 14345 non-null  float64
 13  insV2                 14345 non-null  float64
 14  insV3                 14345 non-null  float64
 15  mrgsV1             

In [536]:
df.to_csv("processesdata.csv")

In [11]:
df

NameError: name 'df' is not defined

In [2]:
import pandas as pd

In [8]:
a=pd.to_datetime("2024-2-22 00:00:00")

In [9]:
a.dt.year()

AttributeError: 'Timestamp' object has no attribute 'dt'

AttributeError: 'Timestamp' object has no attribute 'astype'

In [1]:
import pandas as pd

In [2]:
d=pd.DataFrame()

In [3]:
d

""
